# Introduction to the prompt engineering

**NOTE:** Some parts of this notebook are adaptations of [deeplearning.ai, ChatGPT Prompt Engineering for Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

## Configurations

In [1]:
import os

import json
import pandas as pd
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity
from redlines import Redlines
from IPython.display import display, Markdown, Latex, HTML, JSON

# setup openai
import openai
from dotenv import load_dotenv

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, HTML

from redlines import Redlines
from IPython.display import display, Markdown, Latex, HTML, JSON


pd.set_option('max_colwidth', 800)


def printmd(string):  ###
    display(Markdown(string))  ###

In [2]:
max_tokens = 500
embedding_model = 'text-embedding-ada-002'
model = 'text-davinci-003'

*NOTE*: to use OpenAI diretly, you need to update the following config and API calls slightly.

In [3]:
# NOTE: Set up Azure OpenAI
load_dotenv()
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_CHAT_VERSION") # chatgpt
# openai.api_version = os.getenv("OPENAI_API_VERSION") # davinci
openai.api_key = os.getenv("OPENAI_API_KEY")

## API calls

### ChatGPT calls

In [4]:
def get_completion_chat(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine="chatgpt",
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [5]:
def get_completion_chat_message(messages, model="gpt-3.5-turbo", temperature=0): 

    response = openai.ChatCompletion.create(
        engine="chatgpt",
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

### GPT-3.5 API call

In [6]:
def get_completion(prompt, model=model, temperature=0): 
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        # stop=stop_sequence,
        model=model,
    )
    return response["choices"][0]["text"]

## Prompt principles

- Write a prompt that is as clear and specific as possible

### Use delimitors to separate the different parts of the prompt

- possible delimitors: ```, """, ###, <>, `<tag> </tag>`

Example is from [here](https://news.microsoft.com/source/features/ai/how-ai-makes-developers-lives-easier-and-helps-everybody-learn-to-develop-software/?culture=en-us&country=us)

In [7]:
text = f"""
Ever since Ada Lovelace, a polymath often considered the first computer programmer,
proposed in 1843 using holes punched into cards to solve
mathematical equations on a never-built mechanical computer,
software developers have been translating their solutions to problems into step-by-step instructions
that computers can understand.
"""

prompt_long_summary = f"""
Summarize the text delimited by triple backticks 
into a single sentence.

```{text}```
"""

print(prompt_long_summary)


Summarize the text delimited by triple backticks 
into a single sentence.

```
Ever since Ada Lovelace, a polymath often considered the first computer programmer,
proposed in 1843 using holes punched into cards to solve
mathematical equations on a never-built mechanical computer,
software developers have been translating their solutions to problems into step-by-step instructions
that computers can understand.
```



In [8]:

response = get_completion_chat(prompt_long_summary)

printmd(f"{response}")

Software developers have been translating solutions to problems into step-by-step instructions for computers since Ada Lovelace proposed using punched cards to solve mathematical equations on a mechanical computer in 1843.

Prompt engineering is an iterative process
- make it even shorter summary

In [9]:
text = f"""
Ever since Ada Lovelace, a polymath often considered the first computer programmer,
proposed in 1843 using holes punched into cards to solve
mathematical equations on a never-built mechanical computer,
software developers have been translating their solutions to problems into step-by-step instructions
that computers can understand.
"""

# NOTE that this is added to the prompt
# "Write a sentence that is at most 10 words long."

prompt_short_summary = f"""
Summarize the text delimited by triple backticks 
into a single sentence.
Write a sentence that is at most 10 words long.

```{text}```
"""


In [10]:
diff = Redlines(prompt_long_summary,prompt_short_summary)
display(Markdown("Difference between long and short summary prompt:\n"))
display(Markdown(diff.output_markdown))

Difference between long and short summary prompt:


Summarize the text delimited by triple backticks 
into a single <span style="color:red;font-weight:700;text-decoration:line-through;">sentence.

</span><span style="color:red;font-weight:700;">sentence.
Write a sentence that is at most 10 words long.

</span>```
Ever since Ada Lovelace, a polymath often considered the first computer programmer,
proposed in 1843 using holes punched into cards to solve
mathematical equations on a never-built mechanical computer,
software developers have been translating their solutions to problems into step-by-step instructions
that computers can understand.
```


In [11]:

response = get_completion_chat(prompt_short_summary)
printmd(f"{response}")

Ada Lovelace proposed using punched cards for computing.

### Generate the output in a structure format
- JSON, XML, HTML, etc.

In [12]:
text = f"""
Ever since Ada Lovelace, a polymath often considered the first computer programmer,
proposed in 1843 using holes punched into cards to solve
mathematical equations on a never-built mechanical computer,
software developers have been translating their solutions to problems into step-by-step instructions
that computers can understand.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
Provide the summary in JSON format with the key "summary":

```{text}```
"""

response = get_completion_chat(prompt)

printmd(f"{json.dumps(json.loads(response), indent=4)}")

{
    "summary": "Software developers have been translating solutions to problems into step-by-step instructions for computers since Ada Lovelace proposed using punched cards to solve mathematical equations in 1843."
}

In [13]:
text = f"""
Ever since Ada Lovelace, a polymath often considered the first computer programmer,
proposed in 1843 using holes punched into cards to solve
mathematical equations on a never-built mechanical computer,
software developers have been translating their solutions to problems into step-by-step instructions
that computers can understand.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.

Provide the summary in HTML format.
Format everything as HTML that can be used in a website. 
Place the description in a <div> element.
Give the page the title 'Exexutive Summary'

```{text}```
"""

response = get_completion_chat(prompt)
display(HTML(response))


In [14]:
response

'<div>\n<h1>Executive Summary</h1>\n<p>Software developers have been translating solutions to problems into step-by-step instructions that computers can understand since Ada Lovelace proposed using holes punched into cards to solve mathematical equations on a never-built mechanical computer in 1843.</p>\n</div>'

### Provide examples to model, "Few-shot learning"
- provide good examples to the model

#### An example of generating code summary using linux kernel comment document style
- https://github.com/torvalds/linux/blob/master/include/linux/list.h

In [15]:
# examples function with comment style we want to document
# https://github.com/torvalds/linux/blob/44c026a73be8038f03dbdeef028b642880cf1511/include/linux/list.h#L29
example = """
/**
 * INIT_LIST_HEAD - Initialize a list_head structure
 * @list: list_head structure to be initialized.
 *
 * Initializes the list_head to point to itself.  If it is a list header,
 * the result is an empty list.
 */
static inline void INIT_LIST_HEAD(struct list_head *list)
{
	WRITE_ONCE(list->next, list);
	WRITE_ONCE(list->prev, list);
}
"""

# function that we want to document with a comment format as above example
# https://github.com/torvalds/linux/blob/44c026a73be8038f03dbdeef028b642880cf1511/include/linux/list.h#LL160C1-L167C2
code = """
static inline void list_replace(struct list_head *old,
				struct list_head *new)
{
	new->next = old->next;
	new->next->prev = new;
	new->prev = old->prev;
	new->prev->next = new;
}

"""
prompt = f"""
You are a skilled developer.
You have been asked to review a function deliminated by thriple backticks.
Please review the function and provide maximum two lines of clear, helpful,
and short documentation comment for the function follow the format of the example:

{example}

```{code}```
"""
print(prompt)


You are a skilled developer.
You have been asked to review a function deliminated by thriple backticks.
Please review the function and provide maximum two lines of clear, helpful,
and short documentation comment for the function follow the format of the example:


/**
 * INIT_LIST_HEAD - Initialize a list_head structure
 * @list: list_head structure to be initialized.
 *
 * Initializes the list_head to point to itself.  If it is a list header,
 * the result is an empty list.
 */
static inline void INIT_LIST_HEAD(struct list_head *list)
{
	WRITE_ONCE(list->next, list);
	WRITE_ONCE(list->prev, list);
}


```
static inline void list_replace(struct list_head *old,
				struct list_head *new)
{
	new->next = old->next;
	new->next->prev = new;
	new->prev = old->prev;
	new->prev->next = new;
}

```



In [16]:
response = get_completion_chat(prompt)
print(f" {response}")

 /**
 * list_replace - Replace an old list_head with a new one
 * @old: the old list_head to be replaced
 * @new: the new list_head to replace the old one
 *
 * Replaces the old list_head with the new one by updating the next and prev pointers of the surrounding list_heads.
 */


You can compare the generated comment with the actual documentation done by the developers, [link](https://github.com/torvalds/linux/blob/44c026a73be8038f03dbdeef028b642880cf1511/include/linux/list.h#LL153C1-L159C4).

Generated comments are as good as the actual documentation.

```
/**
 * list_replace - replace old entry by new one
 * @old : the element to be replaced
 * @new : the new element to insert
 *
 * If @old was empty, it will be overwritten.
 */
 ```


### Sentiment analysis
- classify the sentiment of the text

Example is from [here](https://www.imdb.com/title/tt20869502/reviews?sort=curated&dir=desc&ratingFilter=1)

In [17]:
text = f"""
It is very rare for me to review a series before finishing it
but this was so darn delightful that I could not stop myself sharing the good news.
After 3 episodes I wrote that if you are not watching this you are missing out.
Also that Eun-bin Park is simply wonderful as the autistic attorney at the centre of this drama,
coping so movingly with the challenges she faces.
"""
prompt = f"""
What is the sentiment of the following movie review, 
which is delimited with triple backticks?

Review text: '''{text}'''
"""
response = get_completion_chat(prompt)
printmd(f"{response}")



Positive

In [18]:
text = f"""
It's not whether you have autism disorder or not; instead,
it is actually called autism spectrum disorder and every autistic person's character could be
very different from the other. However, the main character of Extraordinary Attorney Woo strengthen
the stereotype of autistic person, who is not familiar with facial expressions of people,
not looking at the other people they are talking to, specially interested in specific things,
has a preference of neat and organized things, has a extremely good memories then ordinary people,
is afraid of noise and sounds, and, last but not least, is a genius.
"""
# additional prompt text: "Provide the sentiment as a number between -1 and 1."
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?
Provide the sentiment as a number between -1 and 1.

Review text: '''{text}'''
"""
response = get_completion_chat(prompt)
printmd(f"{response}")

-0.4

Extract information from the text:

In [19]:
text = f"""
It's not whether you have autism disorder or not; instead,
it is actually called autism spectrum disorder and every autistic person's character could be
very different from the other. However, the main character of Extraordinary Attorney Woo strengthen
the stereotype of autistic person, who is not familiar with facial expressions of people,
not looking at the other people they are talking to, specially interested in specific things,
has a preference of neat and organized things, has a extremely good memories then ordinary people,
is afraid of noise and sounds, and, last but not least, is a genius.
"""

prompt = f"""
What is the character name of the show?

Text: '''{text}'''
"""
response = get_completion_chat(prompt)
printmd(f"{response}")

The character name is Woo.